In [1]:
import requests, json
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, DecimalType, TimestampType
from pyspark.sql.functions import to_timestamp, col

StatementMeta(, 34395f4d-b260-4684-8b04-e6e2f4f385b4, 3, Finished, Available, Finished)

In [2]:
payload = {}
headers= {
  "apikey": "z2Let6TGcnLI6HCDYquelYZ6AaNqckOI"
}

url = "https://api.apilayer.com/exchangerates_data/latest"

res = requests.request("GET", url, headers=headers, data = payload)

StatementMeta(, 34395f4d-b260-4684-8b04-e6e2f4f385b4, 4, Finished, Available, Finished)

In [3]:
ratesJson = json.loads(res.text)

StatementMeta(, 34395f4d-b260-4684-8b04-e6e2f4f385b4, 5, Finished, Available, Finished)

In [4]:
ratesJson = json.loads(res.text)

date = ratesJson["date"]
currencyFrom = ratesJson["base"]
rates = ratesJson["rates"]

data = [(date, currencyFrom, currencyTo, rate) for currencyTo, rate in rates.items()]

StatementMeta(, 34395f4d-b260-4684-8b04-e6e2f4f385b4, 6, Finished, Available, Finished)

In [5]:
schema = StructType([
    StructField("dateString", StringType(), True),
    StructField("currencyFrom", StringType(), True),
    StructField("currencyTo", StringType(), True),
    StructField("rateString", StringType(), True),
])

df = spark.createDataFrame(data, schema)

df = df.withColumn("date", to_timestamp(df.dateString, "yyyy-MM-dd")).withColumn("rate", col("rateString").cast(DecimalType(18,5))).select('date', 'currencyFrom', 'currencyTo', 'rate')

StatementMeta(, 34395f4d-b260-4684-8b04-e6e2f4f385b4, 7, Finished, Available, Finished)

In [6]:
df.write.mode("append").option("overwriteSchema", "True").format("delta").saveAsTable("ExchangeRates")

StatementMeta(, 34395f4d-b260-4684-8b04-e6e2f4f385b4, 8, Finished, Available, Finished)